In [ ]:
import tensorflow as tf 
import nbimporter

from GPT_2 import gpt2

In [ ]:
# Thiết lập phương thức lấy mẫu dữ liệu top k 
def top_k_logits(logits, k):
    if k == 0:
        # không cắn ngắn 
        return logits 
    
    # xây dựng hàm top_k xử lý 
    def _top_k():
        # lấy ra gía trị và chỉ số của phép lấy mẫu top_k 
        # từ kết quả của tensor logits 
        value, _ = tf.nn.top_k(logits, k=k) 
        # lấy giá trị nhỏ nhất của danh sách value (giá trị cuối cùng trong hàng)
        # và thêm 1 trục mới vào . Điều này tạo ra một tensor có cùng số lượng
        # hàng như values nhưng chỉ có một cột.
        min_values = value[:, -1, tf.newaxis]
        # hàm tf.where kiểm tra điều kiện logits < min_values nếu điêu kiện đúng 
        # nó sẽ lấy giá trị thứ 2 tf.ones_like(logits, dtype=logits.dtype) * -1e10,
        # nếu điều kiện sai nó sẽ giữ nguyên logits . Tensor thứ 2 này được tạo ra bằng 
        # cách nhân tensor của các số 1 có cùng hình dnagj với logits với 1e-10
        return tf.where(
            logits < min_values,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )
    # cuối cùng sử dụng tf.cond (conditional) kiểm tra xem k có  = 0
    # nếu k = 0 sẽ trả về logits không thay đổi 
    # nếu  không thì gọi hàm top_k và áp dụng các bước trên 
    return tf.cond(
       tf.equal(k, 0),
       lambda: logits,
       lambda: _top_k(),
    )

In [ ]:
# thiết lập phương thức lấy mẫu tuần tự 
def sample_sequence(*, params, length, start_token=None, batch_size=None, context=None,temperature=1, top_k=0):
    # kiểm tra xem start_tokens = True 
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
        'Chỉ định chính xác 1 trong start_token và bối cảnh'
    # trường hợp còn lại 
    else:
        # kiểm tra điều kiện tương tụ 
        assert context is None, 'Specify exactly one of start_token and context!'
        # tạo một tensor mới bằng cachs sao chép danh sách start token ténor shap[batch_size,1]
        context = tf.fill([batch_size, 1], start_token)

        # tạo một biến length gán cho nó kết quả của length - giá trị nguyên khóa text_length 
        # trong từ điển params 
        length = length - params["text_len"]

    # thiết lập phương thức xủ lý model 
    # token là tensor tokens đầu vào past là tensor chứa các thông tin đã được huấn luyện 
    # ở bước trước đó 
    def step(params, tokens , past=None):
        # kiểm tra xem chế độ percision (độ chính xác) có đang sử dụng kiểu bfloat 16
        if params['percision'] == 'bfloat16':
            # khởi tạo một biến phạm vi 
            with tf.contrib.tpu.bfloat16_scope():
                # thực hiện xử lý gpt model gán kết quả cho biến lm_output
                lm_output = gpt2.model(params=params, X=tokens, past=past, reuse=tf.AUTO_REUSE)

            # ghi vào khóa nhật ký của lm_output giá trị của chính nó nhưng kiểu dữ liệu là float32
            lm_output["logits"] = tf.cast(lm_output["logits"], tf.float32)
        # trường hợp còn lại 
        else:
            # thực hiện tương tự 
            lm_output = lm_output = gpt2.model(params=params, X=tokens, past=past, reuse=tf.AUTO_REUSE)

        # lấy kích thước của tensor lm_output = kích thước của khóa n_vocab trong từ điển params 
        # là kích thước của tập từ vựng 
        logits = lm_output['logits'][:, :, params['n_vocab']]
        # lấy kết quả của từ điển lm_output khóa present là kết quả hiện tại 
        # gán lại cho biến present
        presents = lm_output['present']
        # đặt hình dạng cho tensor presents dựa trên hình dạng mong đợi, được tính toán từ hàm gpt2.past_shape. Điều này cần thiết để đảm bảo rằng tensor
        # presents có hình dạng đúng khi được truyền vào mô hình cho các bước tiếp theo.
        presents.set_shape(gpt2.past_shape(params=params, batch_size=batch_size))
        # trả về 1 từ điển 2 khóa là logits và present 
        return {
            'logits': logits,
            'presents': presents,
        }
    
    # khởi tạo biến phạm vi name_scope là phạm vi hoạt động 
    # có tên là sample_sequence
    with tf.name_scope("sample_sequence"):

        # tạo ra một đầu ra từ step lấy tất acr phần tử từ danh sách trừ phần tử cuối cùng 
        context_output = step(params, context[:, :-1])
        # xây dựng phương thức body là một hàm được lặp đi lặp lại trong quá trình tạo mẫu chuỗi 
        def body(past, prev, output):
            #tạo ra đầu ra tiếp theo từ hàm step 
            # prev[:, tf.newaxis]: Một tensor mới được tạo từ prev bằng cách thêm một chiều mới,
            # giúp chuẩn bị dữ liệu cho bước tiếp theo của quá trình dự đoán.
            next_outputs = step(params, prev[:, tf.newaxis], past=past)
            # trích xuất giá trị đầu ra logits của hàm step trước khi áp dụng hàm softmax
            # next_outputs['logits][:, -1, :] lấy logits cuủa phần tử cuối cùng trong step 
            # Chia các logits cho temperature, một giá trị dương, để kiểm soát sự phân phối của các mẫu được sinh ra. K
            logits = next_outputs['logits'][:, -1, :]  / tf.to_float(temperature)
            # lấy mẫu top_k  Giới hạn logits để chỉ xem xét top_k giá trị cao nhất,
            # giúp tập trung vào các lựa chọn có khả năng cao nhất.
            logits = top_k_logits(logits, k=top_k)
            # lấy mẫu từ phân phối xác suất được định nghĩa bởi logits 
            # Khi hàm tf.multinomial được thực thi, nó sẽ chọn một mẫu từ phân phối xác suất 
            # dựa trên các logits đã cho. Mỗi mẫu được lấy một cách ngẫu nhiên,
            # nhưng với xác suất tương ứng với giá trị của logits
            samples = tf.multinomial(logits, num_samples=1, output_dtype=tf.int32)
            # trả về kết quả  đầu 
            return [
                # nối 2 tensor past và giá trị present trong từ điển next_output theo chiều axis = -2
                tf.concat([past, next_outputs['presents']], axis=-2),
                # bỏ đi chiều thứ 2 trong tensor samples
                tf.squeeze(samples, axis=[1]),
                # nối tensor output với samples 
                tf.concat([output, samples], axis=1),
            ]
        
        # def cond: Hàm điều kiện luôn trả về True, cho phép vòng lặp tiếp tục không giới hạn.
        def cond(*args):
            return True 
        

        # Phần tiếp theo của đoạn mã TensorFlow này sử dụng hàm tf.while_loop 
        # để lặp đi lặp lại một chuỗi các bước tính toán cho đến khi một điều kiện nhất định không còn đúng nữa hoặc đạt đến số lần lặp tối đa.
        _, _, tokens = tf.while_loop(
            # cond=cond: Điều kiện để tiếp tục vòng lặp. Trong trường hợp này, cond luôn trả về True, 
            # nên vòng lặp sẽ tiếp tục cho đến khi đạt đến maximum_iterations.

            # Hàm body chứa logic để thực hiện mỗi lần lặp. 
            # Nó sẽ sử dụng các biến loop_vars và trả về kết quả mới cho mỗi lần lặp.
            cond=cond, body=body,
            # Nó sẽ sử dụng các biến loop_vars và trả về kết quả mới cho mỗi lần lặp.
            # Nó sẽ sử dụng các biến loop_vars và trả về kết quả mới cho mỗi lần lặp.
            maximum_iterations=length,
            # Một danh sách các biến sẽ được truyền vào hàm body và được cập nhật sau mỗi lần lặp.
            loop_vars=[
                context_output['presents'],
                context[:, -1],
                context,
            ],
            # Một danh sách các tf.TensorShape xác định hình dạng không đổi của các
            # tensor trong loop_vars qua mỗi lần lặp. Điều này giúp 
            # TensorFlow kiểm soát hình dạng của các tensor và đảm bảo tính nhất quán.
            shape_invariants=[
                tf.TensorShape(gpt2.past_shape(params=params, batch_size=batch_size)),
                tf.TensorShape([None]),
                tf.TensorShape([None, None]),
            ],
            # Đặt back_prop thành False để ngăn chặn việc tính toán gradient qua vòng lặp này,
            # thường được sử dụng trong quá trình dự đoán hoặc tạo mẫu, 
            # không phải trong quá trình huấn luyện.
            back_prop=False,
        )
        # Cuối cùng, sau khi vòng lặp kết thúc, tensor tokens sẽ được trả về. tokens 
        # chứa chuỗi các phần tử (ví dụ: từ hoặc ký tự) đã được tạo mẫu qua vòng lặp.
        return tokens